In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

In [0]:
class ValidateNPI:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
    self.usecase_id = None
  
  def logger(self, message):
    print(message)
    
  def set_query_string(self, usecase_parm):
    query = f"""WITH t(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase where USECASE_ID = '{usecase_parm}')
update com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed process_table
set process_table.Not_Sent_To_Veeva_Reason = (case  when ISNULL(process_table.Not_Sent_To_Veeva_Reason)= true then 'Invalid NPI Value' else concat(process_table.Not_Sent_To_Veeva_Reason, '|', 'Invalid NPI Value' ) end)
where process_table.usecase_id = '{usecase_parm}'
and process_table.HCP_NPI not in (select identifier from com_us_hub.ref_cust_identifier where identifier_typ = 'NPI' and scd_curr_ind = 'Y')
and CAST(created_date as DATE) >= date_add(current_date, -(select t.hist_retention from t)) 
and SUGGEST_EXTERNAL_ID_VOD__C in (select p.SUGGEST_EXTERNAL_ID_VOD__C from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0));"""
    self.query = query    
    
  def run_query_and_set_sqldf(self, usecase_id):
      self.usecase_id = usecase_id
      query_list = self.query.split(";")
      self.sql_df = []  
      for curr_query in query_list:
          if curr_query.strip() == "":
              continue
          curr_query_with_param = curr_query.replace("Usecase_parm", usecase_id)
          sql_df = sqlContext.sql(curr_query_with_param).collect()
          self.sql_df.extend(sql_df)
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
usecase_id = dbutils.widgets.get("usecase_id")
nb_obj = ValidateNPI()
nb_obj.set_query_string(usecase_parm=usecase_id)
nb_obj.run_query_and_set_sqldf(usecase_id=usecase_id)
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(nb_obj.response)

{'nge_response': {'status': 200, 'body': {'data': [{'num_affected_rows': 0}]}}}